# Parameter-Efficient Fine-tuning of Whisper for MSA Arabic using PEFT & LoRA

This notebook demonstrates how to fine-tune Whisper models for Modern Standard Arabic (MSA) using Parameter-Efficient Fine-Tuning (PEFT) with Low-Rank Adaptation (LoRA). This approach:

1. **Reduces memory usage**: Fine-tune large models with less GPU memory
2. **Faster training**: Only trains 1% of the model parameters
3. **Better generalization**: Prevents catastrophic forgetting
4. **Smaller checkpoints**: Model adapters are ~60MB vs full model ~1.5GB

## 🚀 T4/A100 Optimized Training

This notebook is optimized for **NVIDIA T4** and **A100** GPUs and includes:
- **8-bit quantization** for maximum memory efficiency
- **Mixed precision (FP16)** training for optimal speed
- **Large batch sizes** taking advantage of modern GPU memory
- **Full Common Voice Arabic dataset** for production-quality results

We'll fine-tune Whisper-small on MSA Arabic using the full Common Voice Arabic dataset with LoRA adapters for optimal performance.

## 1. Environment Setup & Installation

First, we'll install the required packages and set up the environment for PEFT training.

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
# 1) Clean out old/conflicting installs
!pip uninstall -y bitsandbytes bitsandbytes-cuda117 bitsandbytes-cuda118 bitsandbytes-cuda121 || true

# 2) Install a known-good, Kaggle-friendly set
!pip install --upgrade pip
!pip install --upgrade accelerate
!pip install "transformers==4.47.0"
!pip install "bitsandbytes==0.45.2"

# 3) (Optional but helpful) make sure CUDA libs are visible in this session
# !python - << 'PY'
# import os, ctypes, sys
# cuda_guess = "/usr/local/cuda/lib64"
# if os.path.isdir(cuda_guess):
#     os.environ["LD_LIBRARY_PATH"] = os.environ.get("LD_LIBRARY_PATH","") + (":" if os.environ.get("LD_LIBRARY_PATH") else "") + cuda_guess
#     try:
#         ctypes.CDLL(cuda_guess + "/libcudart.so")
#         print("✔ CUDA runtime visible via LD_LIBRARY_PATH")
#     except Exception as e:
#         print("⚠ Could not preload libcudart:", e)
# print("LD_LIBRARY_PATH =", os.environ.get("LD_LIBRARY_PATH","(unset)"))
# PY

# 4) Sanity check bitsandbytes can see CUDA
!python -m bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 43.9 MB/s  0:00:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 36.4 MB/s  0:00:09
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 74.4 MB/s  0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 62.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 71.0 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 79.8 MB/s  0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.1 MB/s  0:00:00
  Atte

In [3]:
# Install required packages for PEFT fine-tuning
!pip install --upgrade pip
!pip install -q datasets[audio]==3.6.0 librosa evaluate jiwer gradio
!pip install -q "peft>=0.5.0"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
bigframes 2.8.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.8.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


## 3. Configuration

In [4]:
# Model and training configuration
model_name_or_path = "openai/whisper-small"
language = "Arabic"
task = "transcribe"

# Dataset configuration - Focus on MSA Arabic
dataset_name = "mozilla-foundation/common_voice_11_0"
language_code = "ar"  # Arabic language code

# Training configuration for best performance on MSA
use_full_dataset = True  # Set to True for full Common Voice Arabic training
training_seed = 42  # For reproducibility

# PEFT optimization parameters for best MSA performance
lora_rank = 32  # Optimal rank for Arabic
lora_alpha = 64  # Optimal scaling factor
lora_dropout = 0.05  # Prevent overfitting
target_modules = ["q_proj", "v_proj"]  # Core attention modules for best efficiency

# Training parameters optimized for MSA Arabic
max_train_steps = 4000  # Sufficient steps for MSA convergence
warmup_steps = 500  # Longer warmup for stability
learning_rate = 1e-3  # Optimal PEFT learning rate
batch_size = 16  # Balanced for P100 memory

print(f"🚀 MSA Arabic Training Configuration:")
print(f"   - Dataset: Common Voice Arabic ({dataset_name})")
print(f"   - Language: {language} (MSA)")
print(f"   - Full dataset: {use_full_dataset}")
print(f"   - LoRA rank: {lora_rank}")
print(f"   - Target modules: {target_modules}")
print(f"   - Learning rate: {learning_rate}")
print(f"   - Max steps: {max_train_steps}")
print(f"   - Batch size: {batch_size}")
print(f"   - Random seed: {training_seed}")

🚀 MSA Arabic Training Configuration:
   - Dataset: Common Voice Arabic (mozilla-foundation/common_voice_11_0)
   - Language: Arabic (MSA)
   - Full dataset: True
   - LoRA rank: 32
   - Target modules: ['q_proj', 'v_proj']
   - Learning rate: 0.001
   - Max steps: 4000
   - Batch size: 16
   - Random seed: 42


## 4. Load Common Voice Arabic Dataset

Load the full Common Voice Arabic dataset for MSA fine-tuning. This provides comprehensive coverage of Modern Standard Arabic speech patterns.

In [5]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

language_abbr = "ar" # Replace with the language ID of your choice here!

common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train+validation")
common_voice["test"] = load_dataset(dataset_name, language_abbr, split="test")

print(common_voice)

README.md: 0.00B [00:00, ?B/s]

common_voice_11_0.py: 0.00B [00:00, ?B/s]

languages.py: 0.00B [00:00, ?B/s]

release_stats.py: 0.00B [00:00, ?B/s]

ValueError: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.

## 5. Data Preprocessing and Feature Extraction

Set up the feature extractor, tokenizer, and processor for Whisper, then preprocess the Arabic dataset.

In [ ]:
### Don't use ya gomma run the whole thing

# Take a subset of the dataset for faster testing
train_subset_size = 1000  # You can adjust this number
test_subset_size = 100   # You can adjust this number

common_voice["train"] = common_voice["train"].select(range(min(train_subset_size, len(common_voice["train"]))))
common_voice["test"] = common_voice["test"].select(range(min(test_subset_size, len(common_voice["test"]))))

print("Subset of common_voice dataset:")
print(common_voice)

In [ ]:
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
)

print(common_voice)

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
from transformers import WhisperTokenizer

task = "transcribe"

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language_abbr, task=task)

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

In [ ]:
print(common_voice["train"][0])

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

In [ ]:
common_voice["train"]

## 6. Data Collator Setup

Create a data collator for PEFT training that handles audio features and text labels properly.

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")

In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
      output_dir="whisper-small/test",
      per_device_train_batch_size=8,
      gradient_accumulation_steps=1,
      learning_rate=1e-3,
      warmup_steps=50,
      max_steps=20, # 2000
      gradient_checkpointing=True,
      fp16=True,
      evaluation_strategy="no",  # Disabled evaluation during training
      per_device_eval_batch_size=8,
      predict_with_generate=False,
      generation_max_length=225,
      save_steps=500,
      logging_steps=5,
      report_to=["tensorboard"],
      load_best_model_at_end=True,
      # metric_for_best_model="wer", # Not needed when evaluation_strategy="no"
      greater_is_better=False,
      save_total_limit=20,
      push_to_hub=False,
      remove_unused_columns=False,
      label_names=["labels"],
)

In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


# trainer = Seq2SeqTrainer(
#     args=training_args,
#     model=model,
#     train_dataset=common_voice["train"],
#     eval_dataset=common_voice["test"],
#     data_collator=data_collator,
#     tokenizer=processor.feature_extractor,
#     callbacks=[SavePeftModelCallback],
# )

import numpy as np
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids = np.where(label_ids != -100, label_ids, processor.tokenizer.pad_token_id)

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    wer = metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    tokenizer=processor.tokenizer,        # 👈 fix here
    compute_metrics=compute_metrics,      # 👈 add this
    callbacks=[SavePeftModelCallback],
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()

In [ ]:
peft_model_id = "ziadtarek12/whisper-small-MSA-finetuned"
model.push_to_hub(peft_model_id)

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "kareemali1/whisper-small-MSA-finetuned"
# peft_model_id = "reach-vb/whisper-large-v2-hindi-100steps" # Use the same model ID as before.
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)
model.config.use_cache = True

In [ ]:
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

# Ensure the model is in evaluation mode
model.eval()

# Setup DataLoader and normalizer
eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []

# Optimized evaluation loop
for batch in tqdm(eval_dataloader):
    with torch.no_grad():
        # Move input features to the GPU
        input_features = batch["input_features"].to("cuda")

        # Generate token ids
        generated_tokens = model.generate(
            input_features=input_features,
            forced_decoder_ids=forced_decoder_ids,
            max_new_tokens=255,
        ).cpu().numpy()

        # Prepare label ids
        labels = batch["labels"].numpy()
        labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)

        # Decode predictions and labels
        decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

        predictions.extend(decoded_preds)
        references.extend(decoded_labels)

        # Normalize text for a more robust WER calculation
        normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
        normalized_references.extend([normalizer(label).strip() for label in decoded_labels])

# Compute WER scores
wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

print(f"WER: {wer}")
print(f"Normalized WER: {normalized_wer}")
print(eval_metrics)

In [ ]:
from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
)
from peft import PeftModel, PeftConfig


peft_model_id = "ziadtarek12/whisper-small-MSA-finetuned" # Use the same model ID as before.
language = "ar"
task = "transcribe"
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)

model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = WhisperTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
processor = WhisperProcessor.from_pretrained(peft_config.base_model_name_or_path, language=language, task=task)
feature_extractor = processor.feature_extractor
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
pipe = AutomaticSpeechRecognitionPipeline(model=model, tokenizer=tokenizer, feature_extractor=feature_extractor)


def transcribe(audio):
    with torch.cuda.amp.autocast():
        text = pipe(audio, generate_kwargs={"forced_decoder_ids": forced_decoder_ids}, max_new_tokens=255)["text"]
    return text

transcribe("test_file.mp3")